# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [25]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [26]:
# While loop ensures that the list of cities is always at least 500 unique cities. Defaults to true until code breaks.
while True:
    # List for holding lat_lngs and cities
    lat_lngs = []
    cities = []

    # Create a set of random lat and lng combinations
    lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
    lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
    lat_lngs = zip(lats, lngs)

    # Identify nearest city for each lat, lng combination
    for lat_lng in lat_lngs:
        city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
        # If the city is unique, then add it to a our cities list
        if city not in cities:
            cities.append(city)

    # If statement confirms city count is sufficient, otherwise the loop is rerun.
    if len(cities) >= 500:
        print(f"There were {len(cities)} unique cities in the list.")
        break
    else:
        print(f"There were only {len(cities)} unique cities in the list. Rerunning list.")
        

There were 596 unique cities in the list.


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [27]:
# Perform API calls and print log of each city as calls are being made.

# Set base url and initialize counting number.  Full url will follow that of the one below from API doc.
#      api.openweathermap.org/data/2.5/weather?q={city name}&appid={your api key}
base_url = "http://api.openweathermap.org/data/2.5/weather"
counted_cities = 0

# Initialize empty lists for relevant values.
city_list = []
lat_list = []
lng_list = []
max_temp_list = []
humidity_list = []
cloudiness_list = []
wind_speed_list = []
country_list = []
date_list = []


# Print for human eyes
print("Beginning Data Retrieval")
print("--------------------------------------")


# for loop runs through all cities in cities array.

for city in cities:

    # try ensures that even if city is not in database, the loop continues.
    # query_url is the url to grab data from api.  Response is where that data is stored.
    try:
        query_url = f"{base_url}?q={city}&appid={weather_api_key}"
        response = requests.get(query_url).json()
        
        # All list appends append relevant data to relevant list.
        city_list.append(response["name"])
        lat_list.append(response["coord"]["lat"])
        lng_list.append(response["coord"]["lon"])
        max_temp_list.append(response["main"]["temp_max"])
        humidity_list.append(response["main"]["humidity"])
        cloudiness_list.append(response["clouds"]["all"])
        wind_speed_list.append(response["wind"]["speed"])
        country_list.append(response["sys"]["country"])
        date_list.append(response["dt"])
        
        # Counted cities is a variable solely for the human that must waste as a guideline for how many cities remain.
        # Print the number of cities counted and the name of the city counted.
        counted_cities += 1
        print(f"Processing Record {counted_cities} | {city}")
    
    # If try fails, the except is triggered and the message states simply that the city is not found, then skips it.
    except:
        print(f"City not found.  Skipping the city {city}")
        
    # Sleeping at end of loop ensures program does not go over api limits.
    time.sleep(1)

Beginning Data Retrieval
--------------------------------------
City not found.  Skipping the city taolanaro
Processing Record 1 | kodiak
Processing Record 2 | ushuaia
Processing Record 3 | busselton
Processing Record 4 | bluff
Processing Record 5 | dwarka
Processing Record 6 | beringovskiy
Processing Record 7 | new norfolk
Processing Record 8 | saskylakh
Processing Record 9 | hirara
Processing Record 10 | yellowknife
Processing Record 11 | tuktoyaktuk
Processing Record 12 | palmer
Processing Record 13 | nome
Processing Record 14 | butaritari
Processing Record 15 | cape town
Processing Record 16 | oyama
Processing Record 17 | datong
Processing Record 18 | upernavik
Processing Record 19 | yar-sale
Processing Record 20 | comodoro rivadavia
Processing Record 21 | plettenberg bay
Processing Record 22 | rikitea
Processing Record 23 | half moon bay
Processing Record 24 | taksimo
Processing Record 25 | atuona
Processing Record 26 | mataura
Processing Record 27 | lorengau
Processing Record 28 

KeyboardInterrupt: 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression